# Azure AI Foundry Agent Demo

This notebook demonstrates how to use Azure AI Foundry Agents with the Microsoft Agent Framework.

Based on: [Azure AI Foundry Agents Documentation](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-types/azure-ai-foundry-agent?pivots=programming-language-python)

## Prerequisites

1. Azure CLI installed and authenticated (`az login`)
2. Azure AI Foundry project created
3. Environment variables set:
   - `AZURE_AI_PROJECT_ENDPOINT`
   - `AZURE_AI_MODEL_DEPLOYMENT_NAME`

## Setup: Load Environment Variables

First, we'll load the required environment variables from the `.env` file.

In [1]:
import os
from pathlib import Path
from dotenv import load_dotenv

# Load environment variables from .env file
# Navigate up from Notebooks -> agents -> getting_started -> samples -> python
env_path = "/Users/arturoquiroga/GITHUB/Agent-Framework-Private/python/.env"
load_dotenv(dotenv_path=env_path)

# Verify environment variables are loaded
print(f"Project Endpoint: {os.getenv('AZURE_AI_PROJECT_ENDPOINT', 'Not set')}")
print(f"Model Deployment: {os.getenv('AZURE_AI_MODEL_DEPLOYMENT_NAME', 'Not set')}")

Project Endpoint: https://aq-ai-foundry-sweden-central.services.ai.azure.com/api/projects/firstProject
Model Deployment: gpt-4.1


## Example 1: Basic Agent Creation

The simplest way to create an agent using environment variables.

In [2]:
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

async def basic_agent_example():
    """Create and use a basic Azure AI Foundry agent."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="HelperAgent",
            instructions="You are a helpful assistant."
        ) as agent,
    ):
        result = await agent.run("Hello! Can you introduce yourself?")
        print(f"Agent: {result.text}")

# Run the example
await basic_agent_example()

Agent: Hello! I’m ChatGPT, an AI language model created by OpenAI. My purpose is to assist you with information, answer your questions, help with tasks, or just have a friendly conversation. If you need help with anything—from explaining complex topics to brainstorming ideas or even writing poems—I'm here for you! How can I assist you today?


## Example 2: Explicit Configuration

Instead of using environment variables, you can provide configuration explicitly.

In [3]:
async def explicit_config_example():
    """Create an agent with explicit configuration."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(
            project_endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"],
            model_deployment_name=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
            async_credential=credential,
            agent_name="ExplicitAgent"
        ).create_agent(
            instructions="You are a helpful assistant with explicit configuration."
        ) as agent,
    ):
        result = await agent.run("What's 2 + 2?")
        print(f"Agent: {result.text}")

# Run the example
await explicit_config_example()

Agent: 2 + 2 equals **4**.


## Example 3: Function Tools

Provide custom function tools to Azure AI Foundry agents.

In [4]:
from typing import Annotated
from pydantic import Field
import importlib.util
from pathlib import Path

# Import weather_utils from the azure_ai_real_weather directory (sibling directory)
weather_utils_path = Path("/Users/arturoquiroga/GITHUB/Agent-Framework-Private/python/samples/getting_started/agents") / "azure_ai_real_weather" / "weather_utils.py"
spec = importlib.util.spec_from_file_location("weather_utils", weather_utils_path)
weather_utils = importlib.util.module_from_spec(spec)
spec.loader.exec_module(weather_utils)
get_real_weather = weather_utils.get_real_weather

async def function_tools_example():
    """Use custom function tools with an agent."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="WeatherAgent",
            instructions="You are a helpful weather assistant.",
            tools=get_real_weather
        ) as agent,
    ):
        result = await agent.run("What's the weather like in Toronto?")
        print(f"Agent: {result.text}")

# Run the example
await function_tools_example()

Agent: Weather in Toronto:
- Conditions: clear sky
- Temperature: 17.7°C (feels like 17.1°C)
- Humidity: 58%
- Wind Speed: 6.2 m/s

It's a pleasant and clear day in Toronto!


## Example 4: Code Interpreter

Azure AI Foundry agents support code execution through the hosted code interpreter.

In [5]:
from agent_framework import HostedCodeInterpreterTool

async def code_interpreter_example():
    """Use the hosted code interpreter tool."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="CodingAgent",
            instructions="You are a helpful assistant that can write and execute Python code.",
            tools=HostedCodeInterpreterTool()
        ) as agent,
    ):
        result = await agent.run("Calculate the factorial of 20 using Python code.")
        print(f"Agent: {result.text}")

# Run the example
await code_interpreter_example()

Agent: The factorial of 20 is 2,432,902,008,176,640,000.


## Example 5: Streaming Responses

Get responses as they are generated using streaming.

In [6]:
async def streaming_example():
    """Stream responses as they are generated."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="StreamingAgent",
            instructions="You are a helpful assistant."
        ) as agent,
    ):
        print("Agent: ", end="", flush=True)
        async for chunk in agent.run_stream("Tell me a short story about a robot learning to paint."):
            if chunk.text:
                print(chunk.text, end="", flush=True)
        print()  # New line at the end

# Run the example
await streaming_example()

Agent: In a bustlingIn a bustling city, city, a little a little robot named robot named Pixel spent Pixel spent its days its days sweeping sweeping floors floors in an in an art museum art museum.. Pixel admired Pixel admired the colorful the colorful paintings paintings but could but could only see only see them through them through logical patterns logical patterns and lines and lines of code of code.

.

One quietOne quiet evening, evening, Pixel Pixel found a found a forgotten forgotten paintbrush tucked behind a display. Curious paintbrush tucked behind a display. Curious, it, it dipped dipped the brush the brush into blue paint into blue paint and traced and traced careful careful lines onto lines onto a a scrap of scrap of paper. The paper. The lines formed lines formed a a rectangle—not rectangle—not quite art quite art,, but but not quite not quite nothing nothing.

.

EachEach night, night, Pixel Pixel experimented with experimented with new new colors and shapes colors and s

## Example 6: Using an Existing Agent by ID

If you have an existing agent in Azure AI Foundry, you can use it by providing its ID.

**Note:** Uncomment and replace `<existing-agent-id>` with your actual agent ID to run this example.

In [7]:
from agent_framework import ChatAgent

async def existing_agent_example():
    """Use an existing agent by its ID."""
    # Replace with your actual agent ID
    agent_id = "asst_5UbkTkbNy4qWWQ2iM16z12R8"  # e.g., "asst_abc123..."
    
    async with (
        AzureCliCredential() as credential,
        ChatAgent(
            chat_client=AzureAIAgentClient(
                async_credential=credential,
                agent_id=agent_id
            ),
            instructions="You are a helpful assistant."
        ) as agent,
    ):
        result = await agent.run("Tell me about Microsoft Agent Framework")
        print(f"Agent: {result.text}")

# Uncomment to run the example
await existing_agent_example()

Agent: Certainly! **Microsoft Agent Framework** refers to a set of technologies and software components provided by Microsoft for the creation of interactive, animated characters (called **agents**) that can interact with users, mainly through speech and text. These agents were intended to make software interfaces more friendly, engaging, and accessible.

## Key Points

### Background
- **Introduced:** Microsoft Agent was released in the late 1990s, primarily for Windows operating systems (Windows 98/XP/2000, etc.).
- **Components:** The framework included an API for software developers, character design tools, speech engines, and sample animated characters (like the well-known "Clippy" in Microsoft Office).

### Features
- **Animated Characters**: Characters could perform animations, display emotions, and even move across the screen.
- **Speech Recognition & Synthesis**: Integration with the Microsoft Speech API (SAPI), allowing agents to recognize spoken commands and reply with synth

## Example 7: Creating and Managing Persistent Agents

For more control over agent lifecycle, you can create persistent agents using the Azure AI Projects client.

In [8]:
from azure.ai.projects.aio import AIProjectClient

async def persistent_agent_example():
    """Create and manage a persistent agent."""
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(
            endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], 
            credential=credential
        ) as project_client,
    ):
        # Create a persistent agent
        created_agent = await project_client.agents.create_agent(
            model=os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"],
            name="PersistentAgent",
            instructions="You are a helpful assistant that persists across sessions."
        )
        
        print(f"Created persistent agent with ID: {created_agent.id}")

        try:
            # Use the agent
            async with ChatAgent(
                chat_client=AzureAIAgentClient(
                    project_client=project_client,
                    agent_id=created_agent.id
                ),
                instructions="You are a helpful assistant."
            ) as agent:
                result = await agent.run("Hello! Remember this: my favorite color is blue.")
                print(f"Agent: {result.text}")
                
                # Ask a follow-up question
                result2 = await agent.run("What is my favorite color?")
                print(f"Agent: {result2.text}")
        finally:
            # Clean up the agent
            await project_client.agents.delete_agent(created_agent.id)
            print(f"Deleted agent {created_agent.id}")

# Run the example
await persistent_agent_example()

Created persistent agent with ID: asst_27ii38n4X8MXBQouE6jnmc2S
Agent: Got it! Your favorite color is blue. If you need recommendations or want to talk about anything related to blue, just let me know!
Agent: Got it! Your favorite color is blue. If you need recommendations or want to talk about anything related to blue, just let me know!
Agent: I don’t know your favorite color yet! If you tell me, I’ll remember it for our conversation. What is your favorite color?
Agent: I don’t know your favorite color yet! If you tell me, I’ll remember it for our conversation. What is your favorite color?
Deleted agent asst_27ii38n4X8MXBQouE6jnmc2S
Deleted agent asst_27ii38n4X8MXBQouE6jnmc2S


## Example 8: Multiple Tools Combined

Combine multiple tools including custom functions and code interpreter.

In [9]:
from random import randint

def roll_dice(
    sides: Annotated[int, Field(description="Number of sides on the dice", ge=4, le=100)] = 6
) -> str:
    """Roll a dice with the specified number of sides."""
    result = randint(1, sides)
    return f"You rolled a {result} on a {sides}-sided dice!"

async def multiple_tools_example():
    """Use multiple tools together."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="MultiToolAgent",
            instructions="You are a helpful assistant that can check weather, roll dice, and execute code.",
            tools=[get_real_weather, roll_dice, HostedCodeInterpreterTool()]
        ) as agent,
    ):
        # Query that uses multiple tools
        result = await agent.run(
            "First, roll a 20-sided dice. Then check the weather in Paris. "
            "Finally, use Python to calculate what 15% of 200 is."
        )
        print(f"Agent: {result.text}")

# Run the example
await multiple_tools_example()

Agent: Here are your results:

- You rolled a 5 on a 20-sided dice!
- Weather in Paris: Clear sky, 16.05°C (feels like 15.04°C), 51% humidity, wind speed 3.6 m/s.
- 15% of 200 is 30.


## Example 9: Conversation Threading

Maintain conversation context across multiple interactions using threads.

In [10]:
async def threading_example():
    """Demonstrate conversation threading."""
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="ThreadedAgent",
            instructions="You are a helpful assistant that remembers conversation context."
        ) as agent,
    ):
        # Create a thread for this conversation
        thread = agent.get_new_thread()
        
        # First message
        print("User: My name is ARTURO and I love riding my road bike.")
        result1 = await agent.run(
            "My name is ARTURO and I love riding my road bike.",
            thread=thread,
            store=True
        )
        print(f"Agent: {result1.text}\n")
        
        # Second message - agent should remember name and interest
        print("User: What's my name and what do I love?")
        result2 = await agent.run(
            "What's my name and what do I love?",
            thread=thread,
            store=True
        )
        print(f"Agent: {result2.text}\n")
        
        # Third message - continuing the conversation
        print("User: Can you suggest a book for me based on my interests?")
        result3 = await agent.run(
            "Can you suggest a book for me based on my interests?",
            thread=thread,
            store=True
        )
        print(f"Agent: {result3.text}")

# Run the example
await threading_example()

User: My name is ARTURO and I love riding my road bike.
Agent: Hi Arturo! It’s great to meet another road cycling enthusiast. How long have you been riding, and do you have any favorite routes or destinations you like to explore on your bike?

User: What's my name and what do I love?
Agent: Hi Arturo! It’s great to meet another road cycling enthusiast. How long have you been riding, and do you have any favorite routes or destinations you like to explore on your bike?

User: What's my name and what do I love?
Agent: Your name is Arturo, and you love riding your road bike.

User: Can you suggest a book for me based on my interests?
Agent: Your name is Arturo, and you love riding your road bike.

User: Can you suggest a book for me based on my interests?
Agent: Absolutely, Arturo! Since you love riding your road bike, I recommend **“The Rider” by Tim Krabbé**. It’s a classic among cyclists and beautifully captures the essence of road cycling—from the physical challenge to the emotional hi

## Summary

This notebook demonstrated key features of Azure AI Foundry Agents:

1. **Basic Agent Creation** - Simple agent setup with environment variables
2. **Explicit Configuration** - Direct configuration without environment variables
3. **Function Tools** - Custom functions as tools for agents
4. **Code Interpreter** - Execute Python code through the hosted interpreter
5. **Streaming Responses** - Real-time response streaming
6. **Existing Agents** - Reuse agents by ID
7. **Persistent Agents** - Full lifecycle management with cleanup
8. **Multiple Tools** - Combine different tool types
9. **Conversation Threading** - Maintain context across interactions

## Next Steps

- Explore [Agent Framework Tutorials](https://learn.microsoft.com/en-us/agent-framework/tutorials/overview)
- Learn about [OpenAI ChatCompletion Agents](https://learn.microsoft.com/en-us/agent-framework/user-guide/agents/agent-types/azure-openai-chat-completion-agent)
- Check out more examples in the `samples` directory